# Traducción del dataset de recetas

### **Objetivo**
Tal y como se había mencionado en el primer notebook "0) EDA", el objetivo final de este proyecto es construir una aplicación inteligente que pueda recomendar recetas adecuadas para el contexto del usuario (teniendo en cuenta los ingredientes disponible que tiene y sus posibles deficiencias nutricionales). Para la recomendación, es necesario tener un set de datos de recetas. En este caso, se ha optado por usar el dataset público https://huggingface.co/datasets/mbien/recipe_nlg que contiene más de 2 millones de recetas diferentes. Sin embargo, este set de datos tiene una **desventaja**, está completamente en inglés.

Por lo tanto, el objetivo de este notebook es detallar el proceso de traducción al español y estandarización del dataset de recetas para adaptarlo a nuestro público objetivo. Antes de la traducción, nos centraremos en crear un mapeo entre los ingredientes en inglés y su equivalente en español para agrupar sinónimos y variaciones de un mismo ingrediente bajo el mismo nombre. Esto reduce el número de ingredientes únicos a traducir, mejora la consistencia y evita duplicados debidos a variaciones léxicas.

Por ejemplo, cuando el dataset contiene “tomato” y “roma tomato”, el mapeo los agrupa bajo “tomate”. De este modo, el modelo de traducción no repite trabajo ni genera etiquetas distintas para el mismo ingrediente.

## 1) Imports

In [11]:
import ast
import re
import unicodedata
import torch
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer

## 2) Flujo del proceso

El proceso seguido para la traducción y estandarización del dataset de recetas incluye varias fases:combinando técnicas de preprocesamiento, construcción de diccionarios personalizados y validación de consistencia. A continuación, se describen los pasos principales:

1) `Análisis inicial del dataset`:
Se realizó una revisión exploratoria de los ingredientes en el dataset de recetas, el cual contenía denominaciones en inglés con múltiples variaciones y sinónimos. Por ejemplo, el término onion podía aparecer también como red onion, white onion o shallot. Con el análisis se pudo determinar la necesidad de agrupar términos que estaban relacionados o era el mismo ingrediente bajo un mismo nombre.

2) `Construcción del diccionario de traducción y equivalencias`: Se elaboró un diccionario manual de mapeo entre ingredientes en inglés y su equivalente en español. Este diccionario no solo traduce el término principal, sino que también agrupa variantes y sinónimos.
Por ejemplo:

- `"tomato", "cherry tomato", "roma tomato" → tomate`
- `"onion", "red onion", "shallot" → cebolla`
- `"potato", "russet", "yukon gold" → patata`

3) `Normalización de términos`: Se aplicaron transformaciones textuales básicas para asegurar uniformidad en el dataset:
- Conversión de todos los términos a minúsculas.
- Eliminación de caracteres especiales innecesarios.
- Sustitución directa de cada término por su equivalente en español estandarizado.

4. `Validación del mapeo y traducción usando modelo`: Tras la aplicación del diccionario, se realizaron verificaciones para detectar posibles ingredientes no traducidos o inconsistencias. Esto incluyó:
- Comparación entre los términos originales y los términos traducidos.
- Identificación de ingredientes no cubiertos en el diccionario inicial.
- Iteraciones adicionales para enriquecer el diccionario con nuevos sinónimos.

5. `Traducción automática (Marian MT) de pendientes`:
Todas las palabras/frases que no se han traducido mediante el diccionario se traducen posteriormente con Marian MT.

## 3) Mapeo de ingredientes y preprocesamiento de datos

En esta sección procedemos a mapear diferentes variaciones de ingredientes en inglés a un término único en español. Esto se ha realizado para evitar demasiados ingredientes únicos o demasiadas variaciones del mismo ingrediente dado que el dataset es grande y require considerables recursos computacionales. Mapeando variaciones de ingredientes al mismo término único ayuda a reducir la carga de procesamiento.

Cabe destacar que no el mapeo no es solamente de inglés -> español, sino que también se ha realizado un mapeo español -> español. En el caso de español -> español, es para controlar la consistencia de los términos. Por ejemplo, en el caso de tener sandía y sandia en los ingredientes de diferentes recetas, el mapeo los uniría bajo un único nombre de "sandia".

In [12]:
# Miramos la estructura del set de datos:
recipe_dataset = pd.read_csv(r'..\dataset\Recipes dataset\recipes_dataset.csv')
recipe_dataset.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


A continuación creamos el mapeo de ingredientes:

In [13]:
# Mapeo de ingredientes

en_spa_ingredients = {
    "Tomate": ["tomato", "roma tomato", "cherry tomato", "tomatillo"],
    "Cebolla": ["onion", "red onion", "white onion", "yellow onion", "shallot", "green onion", "spring onion", "scallion"],
    "Patata": ["potato", "new potato", "russet", "yukon gold", "baking potato"],
    "Lechuga/Endivia": ["lettuce", "iceberg", "romaine", "butterhead", "bibb", "cos", "endive", "escarole"],
    "Zanahoria": ["carrot", "baby carrot"],
    "Calabacines": ["zucchini", "courgette", "summer squash"],
    "Pepino": ["cucumber", "english cucumber", "kirby"],
    "Champiñones": ["mushroom", "button mushroom", "cremini", "portobello", "shiitake", "oyster mushroom", "chanterelle", "porcini"],
    "Brocoli": ["broccoli", "broccolini"],
    "Coliflor": ["cauliflower"],

    "Leche": ["milk", "whole milk", "skim milk", "2% milk", "evaporated milk"],
    "Huevos": ["egg", "eggs"],
    "Yogur": ["yogurt", "greek yogurt", "yoghurt"],
    "Queso": ["cheese", "cheddar", "mozzarella", "parmesan", "feta", "gouda", "goat cheese", "blue cheese", "ricotta", "cream cheese", "swiss"],
    "Mantequilla": ["butter", "unsalted butter", "salted butter", "ghee"],

    "Merluza": ["hake"],
    "Gambas/Langostinos": ["shrimp", "prawn", "prawns", "king prawn"],
    "Mix de marisco/molusco": ["seafood mix", "mixed seafood", "clams", "mussels", "oysters", "scallops", "squid", "calamari", "octopus"],
    "Lubina": ["sea bass", "seabass", "branzino", "european seabass"],
    "Salmón": ["salmon"],

    "Plátano": ["banana", "plantain"],
    "Aguacate": ["avocado"],
    "Sandía": ["watermelon"],
    "Limón": ["lemon"],
    "Manzana": ["apple", "granny smith", "gala apple", "fuji apple"],

    "Carne pollo": ["chicken", "chicken breast", "chicken thigh", "chicken leg", "rotisserie chicken", "ground chicken"],
    "Carne cerdo": ["pork", "pork loin", "pork chop", "pork shoulder", "ground pork", "bacon"],
    "Carne vacuno": ["beef", "steak", "ground beef", "sirloin", "ribeye", "chuck", "brisket"],
    "Salchichas": ["sausage", "sausages", "hot dog", "frankfurter", "chorizo", "kielbasa", "bratwurst"],
    "Carne pavo": ["turkey", "ground turkey", "turkey breast", "turkey mince"],
}

spa_spa_ingredients = {
    "Tomate": ["tomate", "jitomate"],
    "Cebolla": ["cebolla", "cebolleta"],
    "Patata": ["patata", "papa"],
    "Lechuga/Endivia": ["lechuga", "endivia", "escarola"],
    "Zanahoria": ["zanahoria"],
    "Calabacines": ["calabacin", "calabacines", "zucchini"],
    "Pepino": ["pepino"],
    "Champiñones": ["champiñon", "champiñones", "seta", "hongos", "portobello", "shiitake"],
    "Brocoli": ["brocoli"],
    "Coliflor": ["coliflor"],
    "Leche": ["leche"],
    "Huevos": ["huevo", "huevos"],
    "Yogur": ["yogur"],
    "Queso": ["queso"],
    "Mantequilla": ["mantequilla", "ghee"],
    "Merluza": ["merluza"],
    "Gambas/Langostinos": ["gamba", "gambas", "langostino", "langostinos", "camarón", "camaron", "camarones"],
    "Mix de marisco/molusco": ["marisco", "molusco", "almeja", "mejillon", "mejillón", "ostras", "calamar", "pulpo"],
    "Lubina": ["lubina"],
    "Salmón": ["salmon", "salmón"],
    "Plátano": ["platano", "plátano", "banana", "banano"],
    "Aguacate": ["aguacate", "palta"],
    "Sandía": ["sandia", "sandía"],
    "Limón": ["limon", "limón"],
    "Manzana": ["manzana"],
    "Carne pollo": ["pollo"],
    "Carne cerdo": ["cerdo"],
    "Carne vacuno": ["vacuno", "ternera", "res"],
    "Salchichas": ["salchicha", "salchichas"],
    "Carne pavo": ["pavo"],
}

Después de haber definido el mapeo, procedemos a construir funciones de preprocesamiento de datos. En resumen se realiza:

- Parseo NER: Convierte la columna NER (que puede venir como string representando una lista) a una lista real de frases por fila.
- Normalización a nivel de frase: Aplica norm(...) a cada frase para unificar el formato (minúsculas, sin acentos, limpieza de símbolos y espacios). No se divide en palabras; se conserva cada frase completa para mantener contexto como “cherry tomato”.
- Unificar las variaciones en una única palabra:
Se utiliza el mapeo definido previamente para unificar las variaciones de diferentes ingredientes en inglés o español a una única palabra en español

**Resultado**: se obtiene una base normalizada de términos NER y un primer mapeo automático hacia etiquetas canónicas en español.

In [14]:
# Funciones normalizadoras:
def strip_accents(s: str) -> str:
    """
    Elimina acentos de una cadena y normaliza la forma Unicode
    """
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")
    return unicodedata.normalize("NFKC", s)

def norm(s: str) -> str:
    """
    Función normalizadora de texto 
    Operaciones:
      - Convierte a minúsculas.
      - Quita acentos (utiliza la función definida previamente)
      - Recorta espacios en extremos.
      - Sustituye todo lo que no sea [a-z0-9, espacio, -, /, +] por espacio.
      - Colapsa múltiples espacios en uno solo.
    """    
    s = strip_accents((s or "").strip().lower())
    s = re.sub(r"[^a-z0-9\s\-\/\+]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s


# Si el valor de la columna NER viene como cadena (por ejemplo, "[ 'cherry tomato', 'red onion' ]"), se usa ast.literal_eval para transformarlo en una lista real de strings.
# Después se usa la función normalizadora
recipe_dataset["NER"] = recipe_dataset["NER"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
recipe_dataset["NER_terms"] = recipe_dataset["NER"].apply(
    lambda phrases: [norm(p) for p in phrases if p]
)

# Funciones que crean la arquitectura del sistema de mapeo
keywords_eng = {norm(kw): cls for cls, kws in en_spa_ingredients.items() for kw in kws}
keywords_spa = {norm(kw): cls for cls, kws in spa_spa_ingredients.items() for kw in kws}

def rule_map_en(term: str):
    """
    Asigna una clase canónica (en español) a un término EN normalizado usando reglas simples.

    Regla de matching:
      1) Coincidencia exacta con 'keywords_eng'.
      2) Si no hay exacta, coincidencia por inclusión de subcadena (kw in term).
    """
    t = norm(term)
    if not t: return None
    if t in keywords_eng:
        return keywords_eng[t]
    for kw, cls in keywords_eng.items():
        if kw and kw in t:
            return cls
    return None

def rule_map_es(term: str):
    """
    Asigna una clase canónica (ES) a un término ES normalizado usando reglas simples.

    Regla de matching:
      1) Coincidencia exacta con 'keywords_spa'.
      2) Si no hay exacta, coincidencia por inclusión de subcadena (kw in term).
    """
    t = norm(term)
    if not t: return None
    if t in keywords_spa:
        return keywords_spa[t]
    for kw, cls in keywords_spa.items():
        if kw and kw in t:
            return cls
    return None

# Se construye un conjunto de términos únicos en el dataset de recetas.
unique_terms = sorted({t for terms in recipe_dataset["NER_terms"] for t in terms if t})
print("Términos de ingredientes únicos en las recetas:", len(unique_terms))


# Se traducen directamente las palabras que son encontradas en el diccionario de mapeo ing-esp, el resto serán traducidas por un modelo NLP.
eng_rule_map = {}
unknown_en = []
for term in unique_terms:
    cls = rule_map_en(term)
    if cls:
        eng_rule_map[term] = cls
    else:
        unknown_en.append(term)

print("Palabras traducidas usando el mapeo ing-esp:", len(eng_rule_map), " | Palabras por traducir, dado que no aparecen en el mapeo:", len(unknown_en))

Términos de ingredientes únicos en las recetas: 196018
Palabras traducidas usando el mapeo ing-esp: 43584  | Palabras por traducir, dado que no aparecen en el mapeo: 152434


## 4) Traducción de los ingredientes de las recetas

Previamente hemos construido el vocabulario de términos NER únicos del dataset de recetas y se han mapeado automáticamente todas los términos que aparecían en el diccionario ing-esp hacia una clase canónica en español.
Para los ingredientes que no quedaron cubiertos por el diccionario (pendientes), se utilizará el modelo `Helsinki-NLP/opus-mt-en-es` para la traducción. Es un sistema de traducción automática neuronal basado en Marian MT.
Marian MT es una implementación eficiente de Transformers encoder–decoder para traducción automática. El modelo Helsinki-NLP/opus-mt-en-es está entrenado sobre OPUS, un gran conjunto de corpora paralelos multifuente. Utiliza subpalabras para manejar vocabularios extensos y es ampliamente usado para traducciones de inglés a español en dominios generales.

In [15]:
# Traducimos todos los términos que no han sido mapeados directamente previamente
mt_name = "Helsinki-NLP/opus-mt-en-es"
device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

mt_tok = MarianTokenizer.from_pretrained(mt_name)
mt_model = MarianMTModel.from_pretrained(mt_name).to(device)
mt_model.eval()

def translate_batch(texts, src_max_len=256, max_new_tokens=128):
    if not texts:
        return []
    with torch.inference_mode():
        inputs = mt_tok(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=src_max_len,
        ).to(device)
        outputs = mt_model.generate(
            **inputs,
            num_beams=1,          
            do_sample=False,
            max_new_tokens=max_new_tokens,
            use_cache=True,
        )
        return mt_tok.batch_decode(outputs, skip_special_tokens=True)

BATCH = 96 if device == "cuda" else 24
en2es = {}
total = len(unknown_en)
for i in range(0, total, BATCH):
    chunk = unknown_en[i:i+BATCH]
    trans = translate_batch(chunk)
    for src, tgt in zip(chunk, trans):
        en2es[src] = tgt
    print(f"Progreso: {min(i+BATCH, total)}/{total} traducidos")

print("Cantidad de términos traducidos:", len(en2es))

Using device: cuda


c:\Users\oscar.xu\AppData\Local\anaconda3\envs\tfm_env_final\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Progreso: 96/152434 traducidos
Progreso: 192/152434 traducidos
Progreso: 288/152434 traducidos
Progreso: 384/152434 traducidos
Progreso: 480/152434 traducidos
Progreso: 576/152434 traducidos
Progreso: 672/152434 traducidos
Progreso: 768/152434 traducidos
Progreso: 864/152434 traducidos
Progreso: 960/152434 traducidos
Progreso: 1056/152434 traducidos
Progreso: 1152/152434 traducidos
Progreso: 1248/152434 traducidos
Progreso: 1344/152434 traducidos
Progreso: 1440/152434 traducidos
Progreso: 1536/152434 traducidos
Progreso: 1632/152434 traducidos
Progreso: 1728/152434 traducidos
Progreso: 1824/152434 traducidos
Progreso: 1920/152434 traducidos
Progreso: 2016/152434 traducidos
Progreso: 2112/152434 traducidos
Progreso: 2208/152434 traducidos
Progreso: 2304/152434 traducidos
Progreso: 2400/152434 traducidos
Progreso: 2496/152434 traducidos
Progreso: 2592/152434 traducidos
Progreso: 2688/152434 traducidos
Progreso: 2784/152434 traducidos
Progreso: 2880/152434 traducidos
Progreso: 2976/152434

Después de aplicar tanto el mapeo de inglés -> español y traducir con modelo Marian-MT los términos para los cuales no se han encontrado un mapeo al español, se realiza un último paso: se une el mapeo procedente de traducción automática (guardado en `en2es`) con el mapeo por reglas/diccionario (`eng_rule_map`).
Si hay conflictos, gana `eng_rule_map` (porque en {**A, **B} las claves de B sobrescriben a A).

Se crea una nueva columna llamada "NER_terms_es" donde obtendremos los ingredientes de la receta traducidos al español.

In [16]:
full_mapping = {**en2es, **eng_rule_map}  

def map_list_to_spanish(terms, mapping):
    return [mapping.get(t, t) for t in (terms or [])]

recipe_dataset["NER_terms_es"] = recipe_dataset["NER_terms"].apply(
    lambda ts: map_list_to_spanish(ts, full_mapping)
)

Miramos el dataset otra vez para ver los resultados de traducción:

In [17]:
recipe_dataset.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER,NER_terms,NER_terms_es
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[brown sugar, milk, vanilla, nuts, butter, bit...","[brown sugar, milk, vanilla, nuts, butter, bit...","[azúcar moreno, Leche, vainilla, frutos secos,..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[beef, chicken breasts, cream of mushroom soup...","[beef, chicken breasts, cream of mushroom soup...","[Carne vacuno, Carne pollo, Champiñones, Crema..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[frozen corn, cream cheese, butter, garlic pow...","[frozen corn, cream cheese, butter, garlic pow...","[Maíz congelado, Queso, Mantequilla, ajo en po..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[chicken, chicken gravy, cream of mushroom sou...","[chicken, chicken gravy, cream of mushroom sou...","[Carne pollo, Carne pollo, Champiñones, Queso]"
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[peanut butter, graham cracker crumbs, butter,...","[peanut butter, graham cracker crumbs, butter,...","[Mantequilla, migajas de galleta graham, Mante..."


Por último guardamos el dataset en formato parquet y csv.

In [ ]:
recipe_dataset.to_parquet("recipes_dataset_translated.parquet", engine="pyarrow", index=False)
recipe_dataset.to_csv("recipes_dataset_translated.csv", index=False, encoding="utf-8-sig")